In [ ]:
# Imports
import numpy as np, matplotlib.pyplot as plt, pandas as pd, \
    re, joblib as jl, statsmodels.stats, statsmodels.stats.multitest, datetime
from tqdm.auto import tqdm

In [ ]:
# One of: '1-human', '2-human', 'CO2', '6-human'
#EXPERIMENT = 'CO2'

# One of: 'LANDINGS' (all EXPERIMENTs), 'OCCUPANCY' (6-human only)
#MODE = 'LANDINGS'

# One of: False, True
#USE_ABSOLUTE_COUNTS = True

### Load data (1-human, 2-human, CO2)

In [ ]:
def getDataMisc():
    fname = 'data_raw/data_AllC.csv'
    # Load data
    data = pd.read_csv(fname).T.reset_index().iloc[:, 1:]
    # Change column names
    data.columns = ['landing_count', 'experiment', 'participant_name', 'participant_position', 'experiment_date']
    # Convert participant name
    participants1human = data.loc[data.experiment=='1-human', 'participant_name'].unique().tolist()
    participants2human = data.loc[data.experiment=='2-human', 'participant_name'].unique().tolist()
    participantsCO2 = data.loc[data.experiment=='CO2', 'participant_name'].unique().tolist()
    # -- Map indices
    data.loc[data.experiment=='1-human', 'participant_id'] = [
        participants1human.index(x) for x in data.loc[data.experiment=='1-human', 'participant_name']]
    data.loc[data.experiment=='2-human', 'participant_id'] = [
        participants2human.index(x) for x in data.loc[data.experiment=='2-human', 'participant_name']]
    data.loc[data.experiment=='CO2', 'participant_id'] = [
        participantsCO2.index(x) for x in data.loc[data.experiment=='CO2', 'participant_name']]
    data.participant_id = data.participant_id.astype(int)
    # Remember mapping from participant id to participant name
    participantNames = {}
    for exp in ['1-human', '2-human', 'CO2']:
        participantNames[exp] = {}
        for pid in data[(data.experiment==exp)].participant_id.unique():
            correspondingName = data[(data.experiment==exp)&(
                data.participant_id==pid)].participant_name.unique()
            assert(len(correspondingName) == 1)
            participantNames[exp][int(pid)] = correspondingName[0]
    # Convert data types
    data.landing_count = data.landing_count.astype(int)
    # Make compatible with 6-human analysis by creating a start/end row for each landing
    newdata = []
    for ri, r in data.iterrows():
        # Add 0-duration trajectory to day/position pair
        newdata.append((r.experiment, r.experiment_date, 
                        r.participant_position, r.participant_id, 
                        0, 0, 0, 2020))
        newdata.append((r.experiment, r.experiment_date, 
                        r.participant_position, r.participant_id, 
                        r.landing_count, 0, 0, 2020))
        # Add landings individually
        for landing_id in range(r.landing_count):
            for frame_rel in [0, 1]:
                newdata.append((r.experiment, r.experiment_date, 
                                r.participant_position, r.participant_id, 
                                1 + landing_id, frame_rel, frame_rel, 2020))
            landing_id += 1
    newdata = pd.DataFrame(newdata, columns=[
        'experiment', 'experiment_date', 'participant_position', 'participant_id', 
        'landing_id', 'frame_rel', 'frame', 'experiment_year'])
    # Subset to requested experiment
    newdata = newdata[newdata.experiment == EXPERIMENT].reset_index()
    # Construct assigned placement dictionary
    personAssignments = {}
    for _d in data.experiment_date.unique():
        for _p in data.participant_position.unique():
            # Retrieve participant at this position on this day
            partid = data.participant_id[(data.experiment == EXPERIMENT)&(
                data.experiment_date==_d)&(data.participant_position==_p)].values
            if partid.size > 0:
                if _d not in personAssignments:
                    personAssignments[_d] = {}
                personAssignments[_d][int(_p)] = partid[0]
    # Assign back to 'data' variable name
    return newdata, personAssignments, participantNames[EXPERIMENT]

### Load data (6-human)

In [ ]:
def getData6human():
    # Specify source data file
    fname = 'data_raw/tracking-predictions-aggregated-startendonly-filtered-50_10_20_0_20.csv'
    
    # Read 2022 data
    data = pd.read_csv(fname)
    data = data[data.experiment_year == 2022]
    data = data.loc[:, ['experiment_year', 'experiment_date', 'participant_position', 
        'landing_id', 'frame_rel', 'frame', 'x', 'y']]
    data = data[~data.experiment_date.isin(['04-07-2022', '04-14-2022'])]

    # Parse position index integer from participant_position
    data.loc[:, 'position_id'] = [int(re.search('[0-9]+', x).group(0)) for x in data.participant_position]
    
    # Look up what person was at each position
    personAssignments = {}
    newDate = None
    personAssignmentsRaw = pd.read_excel('data_raw/human_positions.xlsx', header=None).values
    for row in personAssignmentsRaw:
        if isinstance(row[0], datetime.datetime):
            newDate = row[0]
        elif str(row[0]).isnumeric():
            position, human = row[:2]
            newDateStr = newDate.strftime('%m-%d-%Y')
            if newDateStr in personAssignments:
                personAssignments[newDateStr][position] = human
            else:
                personAssignments[newDateStr] = {position: human}
    
    # Get participant name strings
    participantNames = {}
    for pid in np.unique(np.hstack([list(personAssignments[x].values()) for x in personAssignments])):
        participantNames[pid] = 'Human {}'.format(pid)
    
    # Convert position index and day to person index
    data.loc[:, 'participant_id'] = [personAssignments[_d][_p] for _p, _d in zip(
        data.position_id, data.experiment_date)]
    
    # Set experiment
    data.loc[:, 'experiment'] = '6-human'
    
    # Done
    return data, personAssignments, participantNames

In [ ]:
getData = getData6human if EXPERIMENT == '6-human' else getDataMisc

data, personAssignments, participantNames = getData()

### Test null hypothesis

In [ ]:
def computeLandingStatistics(data, shufflePositions=False, 
        locationWeights=None, mode='', personAssignments=None):
    # Compute landing durations
    dataAgg = data.groupby([
        'experiment_year', 'experiment_date', 'participant_position', 
        'landing_id']).agg(['min', 'max'])
    dataAgg.columns = ['_'.join(col).strip() for col in dataAgg.columns.values]
    dataAgg.loc[:, 'duration'] = dataAgg.frame_max - dataAgg.frame_min
    dataAgg.loc[:, 'duration_rel'] = dataAgg.frame_rel_max - dataAgg.frame_rel_min
    dataAgg = dataAgg.reset_index()
    dataAgg.head()
    
    # Create score column that is either fixed per trajectory (1), or set to duration
    if mode == 'OCCUPANCY':
        if EXPERIMENT == '6-human':
            dataAgg.loc[:, 'weight'] = dataAgg.duration
        else:
            # No duration data analyzed for other experiments, do not allow this config combination
            raise Exception('Occupancy duration only analyzed for 6-human experiment.')
    elif mode == 'LANDINGS':
        if EXPERIMENT == '6-human':
            dataAgg.loc[:, 'weight'] = 1.0
        else:
            # NOTE: Only for CO2/1-human/2-human data:
            #       The duration is set to either 1, or set to 0 for one "fake" landing per 
            #       experimental day/position. This facilitates analysis. Note we are still 
            #       counting landings here, not measuring occupancy.
            dataAgg.loc[:, 'weight'] = dataAgg.duration
    else:
        raise Exception('Invalid analysis mode')
    
    # Compute number of landings by day
    dataAgg.loc[:, 'total_landings_by_day'] = dataAgg.groupby([
        'experiment_year', 'experiment_date']).transform('sum').weight.astype(float)

    # Parse position index integer from participant_position
    dataAgg.loc[:, 'position_id'] = [int(re.search('[0-9]+', x).group(0)) for x in dataAgg.participant_position]

    # Shuffle?
    if shufflePositions and locationWeights is not None:
        for p in dataAgg.experiment_date.unique():
            # Randomly divide landings from day X according to overall location biases
            dataAgg.loc[dataAgg.experiment_date==p, 'position_id'] = \
                np.random.choice(locationWeights.index.values, p=locationWeights.values, replace=True,
                    size=(dataAgg.experiment_date==p).sum())
        
    # Compute number of landings by individual/position for each day
    dataAgg.loc[:, 'position_landings_by_day'] = dataAgg.groupby([
        'experiment_year', 'experiment_date', 'position_id']).transform('sum').weight.astype(float)

    # Compute percent landings by individual/position for each day
    dataAgg.loc[:, 'percent_landings_'] = 100 * dataAgg.position_landings_by_day / \
        dataAgg.total_landings_by_day
    
    # In regular mode, we compute landing differences as percentage landing differences.
    # If USE_ABSOLUTE_COUNTS is enabled, the landing differences are displayed as absolute 
    # landing counts (though the column name remains 'percent_landings'). However, even 
    # if USE_ABSOLUTE_COUNTS is enabled, the biases for each position (used to simulate 
    # the null hypothesis) will still be based on landing percentages.
    if USE_ABSOLUTE_COUNTS:
        dataAgg.loc[:, 'percent_landings'] = dataAgg.position_landings_by_day
    else:
        dataAgg.loc[:, 'percent_landings'] = dataAgg.loc[:, 'percent_landings_']

    # Convert position index and day to person index
    dataAgg.loc[:, 'participant_id'] = [personAssignments[_d][_p] for _p, _d in zip(
        dataAgg.position_id, dataAgg.experiment_date)]

    # Get landing statistics only (no trajectory data)
    dataLandings = dataAgg.loc[dataAgg.landing_id==0, [
        'experiment_date', 'percent_landings', 'percent_landings_', 'participant_id', 
            'position_id', 'total_landings_by_day']].reset_index(drop=True)
    
    # Get mean percent landings by person
    dataLandings.loc[:, 'percent_landings_mean_participant'] = dataLandings.groupby(
        'participant_id').transform('mean').percent_landings_

    # Get mean percent landings by position
    dataLandings.loc[:, 'percent_landings_mean_position'] = dataLandings.groupby(
        'position_id').transform('mean').percent_landings_
    
    # Sort by participant
    dataLandingsSortedParticipants = dataLandings.sort_values(
        'percent_landings_mean_participant', ascending=False)

    # Sort by position
    dataLandingsSortedPositions = dataLandings.sort_values(
        'percent_landings_mean_position', ascending=False)

    # Get participant ranking
    participantRanking = dataLandings.groupby(
        'participant_id').first().percent_landings_mean_participant.rank(ascending=False)

    # Get position ranking
    positionRanking = dataLandings.groupby(
        'position_id').first().percent_landings_mean_position.rank(ascending=False)
    
    # Return results
    return dataAgg, dataLandings, dataLandingsSortedParticipants, \
        dataLandingsSortedPositions, participantRanking, positionRanking

In [ ]:
# Get mean-based ranking of participants
data, personAssignments, participantNames = getData()
dataAgg, dataLandings, dataLandingsSortedParticipants, dataLandingsSortedPositions, \
    participantRanking, positionRanking = computeLandingStatistics(
        data, shufflePositions=False, mode=MODE, personAssignments=personAssignments)
participantsSorted = dataLandingsSortedParticipants.participant_id.unique()

In [ ]:
# Randomization test for 5000 (location-weighted) random trajectories
def getRankings(mode):
    data, personAssignments, participantNames = getData()
    
    # Compute location weights
    dataLandings = computeLandingStatistics(
        data, shufflePositions=False, mode=mode, personAssignments=personAssignments)[1]
    weightsPositions = dataLandings.groupby(
        'position_id').sum().percent_landings_mean_position.copy()
    weightsPositions = weightsPositions / weightsPositions.sum()
    
    # Compute shuffled rankings
    participantRankings = []
    for k in range(50):
        print(k)
        dataAgg, dataLandings, dataLandingsSortedParticipants, dataLandingsSortedPositions, \
            participantRanking, positionRanking = computeLandingStatistics(
                data, shufflePositions=True, locationWeights=weightsPositions, 
                mode=mode, personAssignments=personAssignments)
        participantRankings.append(dataLandings.groupby('participant_id').mean().percent_landings)
    return participantRankings

# Run in parallel
participantRankingsL = jl.Parallel(n_jobs=58)(jl.delayed(
    getRankings)(MODE) for i in tqdm(range(100)))

In [ ]:
# Non-shuffled rankings
participantRankingsTrueL = computeLandingStatistics(
    data, shufflePositions=False, mode=MODE, personAssignments=personAssignments)[1]
participantRankingsTrueL = participantRankingsTrueL.groupby('participant_id').mean().percent_landings

In [ ]:
# Helper function for getting landing % difference
def getLandingDiff(x, pA, pB):
    vA, vB = 0, 0
    if pA in x.index:
        vA = x.loc[pA]
    if pB in x.index:
        vB = x.loc[pB]
    return vA - vB

In [ ]:
def randomizationTest(participantsSorted, participantRankings, participantRankingsTrue):
    numDts = 8
    pvals = {}
    for dti, dt in enumerate(range(1, 1 + numDts)):
        for c in range(len(participantsSorted) - dt):
            pA = participantsSorted[c]
            pB = participantsSorted[c+dt]

            randvals = [getLandingDiff(x, pA, pB) for y in participantRankings for x in y]
            trueval = getLandingDiff(participantRankingsTrue, pA, pB)
            pvals[(pA, pB)] = (np.mean(randvals > trueval), randvals, trueval)
    
    adj = statsmodels.stats.multitest.multipletests(
        [pvals[x][0] for x in pvals], alpha=0.05, method='fdr_bh')[1]
    
    npvals = {}
    for xi, x in enumerate([x for x in pvals]):
        npvals[x] = (pvals[x][0], pvals[x][1], pvals[x][2], adj[xi])
    
    return npvals

testsL = randomizationTest(participantsSorted, participantRankingsL, participantRankingsTrueL)

In [ ]:
for measure in ['landings', ]:
    plottingData = []

    tests = testsL
    numDts = participantsSorted.size - 1

    # Embed metadata such as participant plot ranking and names
    for participantID in participantNames:
        plottingData.append((
            measure, 'name_and_ranking', participantID, -1, 
            0, 0, 0, 0, 
            participantNames[participantID],
            np.argwhere(np.array(participantsSorted) == participantID)[0, 0]
        ))
    
    # Determine min/max range across all pairings
    mn = np.min([np.min(tests[x][1]) for x in tests])
    mx = np.max([np.max(tests[x][1]) for x in tests])

    for dti, dt in enumerate(range(1, 1 + numDts)):
        for c in range(len(participantsSorted) - dt):
            pA = participantsSorted[c]
            pB = participantsSorted[c+dt]
            
            pval, randvals, trueval, pvaladj = tests[(pA, pB)]
                        
            # Compute histogram for this pairing
            pval, randvals, trueval, pvaladj = tests[(pA, pB)]
            counts, edges = np.histogram(randvals, bins=100, range=(mn, mx))
            bins = edges[1:] * 0.5 + edges[:edges.size-1] * 0.5
            
            for binn, count in zip(bins, counts):
                plottingData.append((measure, 'randomized', pA, pB, binn, int(
                    count), pval, pvaladj, '', -1))
            plottingData.append((measure, 'observed', pA, pB, trueval, 
                1, pval, pvaladj, '', -1))

    plottingData = pd.DataFrame(plottingData, columns=['measure', 'randomized_or_observed', 'human_or_stim_A', 
        'human_or_stim_B', 'difference_histogram_bin', 'count', 'pval_unadj', 'pval_adj', 
        'human_or_stim_name', 'human_or_stim_ranking'])

    plottingData.to_csv('Landings_Permutation_Test_{}_{}_{}.csv'.format(
            EXPERIMENT, MODE, 'absolutecounts' if USE_ABSOLUTE_COUNTS else 'percentages'), index=False)

### Plot

In [ ]:
# Ensure that the code below runs with only CSV data by clearing all previously-defined variables
%reset_selective -f ^(?!EXPERIMENT$|MODE$|USE_ABSOLUTE_COUNTS$).*

In [ ]:
import numpy as np, matplotlib.pyplot as plt, pandas as pd

In [ ]:
plottingData = pd.read_csv('Landings_Permutation_Test_{}_{}_{}.csv'.format(
    EXPERIMENT, MODE, 'absolutecounts' if USE_ABSOLUTE_COUNTS else 'percentages'))
plottingData.head(10)

In [ ]:
# Recover participant sorting from CSV data
srt = plottingData[plottingData.randomized_or_observed == 'name_and_ranking'].groupby('human_or_stim_A').first()
stims = srt.index
stimsR = srt.human_or_stim_ranking
stimsN = srt.human_or_stim_name

participantsSorted = [-1 for i in range(len(stims))]
participantNames = {}

for s, r, n in zip(stims, stimsR, stimsN):
    participantsSorted[r] = int(s)
    participantNames[s] = n

participantsSorted = np.array(participantsSorted)
participantsSorted

In [ ]:
def plotRankingTests(plottingData, MODE):
    numDts = participantsSorted.size - 1

    fig, ax = plt.subplots(numDts, len(participantsSorted) - 1, figsize=(14, 1.75 * numDts), facecolor='white')
    
    if ax.ndim == 1:
        ax = np.array([ax,]).T
        print(ax.shape)
    
    for dti, dt in enumerate(range(1, 1 + numDts)):
        for c in range(len(participantsSorted) - dt):
            pA = participantsSorted[c]
            pB = participantsSorted[c+dt]
            
            # Get data for plotting
            d = plottingData.loc[(
                plottingData.human_or_stim_A==pA)&(
                plottingData.human_or_stim_B==pB)&(
                plottingData.randomized_or_observed=='randomized'), [
                    'difference_histogram_bin', 'count', 'pval_adj']].values
            bins, counts, pvaladj = d[:, 0], d[:, 1], d[0, 2]
            
            trueval = plottingData.loc[(
                plottingData.human_or_stim_A==pA)&(
                plottingData.human_or_stim_B==pB)&(
                plottingData.randomized_or_observed=='observed'), [
                    'difference_histogram_bin', ]].values[0, 0]
            
            # plot
            ax[dti, c].fill_between(bins, counts, color='#666666')
            ax[dti, c].axvline(trueval, c='red')
            
            ax[dti, c].set_title('#{} vs. #{}'.format(
                participantNames[pA], participantNames[pB]), size=10)
            
            ax[dti, c].text(0.05, 0.95, (
                'p={:.2E}' if (pvaladj > 1e-12 and pvaladj < 0.01) else 'p={:.3f}').format(pvaladj),
                 horizontalalignment='left',
                 verticalalignment='top',
                 transform = ax[dti, c].transAxes, 
                 fontsize=12)

            if c == 0:
                ax[dti, c].set_ylabel('Occurrences')
            else:
                ax[dti, c].yaxis.set_ticklabels([])
            if c == len(participantsSorted) - dt - 1:
                ax[dti, c].set_xlabel('{} {}\nDifference\n(First - Second)'.format(
                    'Landing' if MODE == 'LANDINGS' else 'Occupancy',
                    'Percentage' if not USE_ABSOLUTE_COUNTS else 'Count'))
            else:
                ax[dti, c].xaxis.set_ticklabels([])
                
            # Set Y axis to same scale everywhere
            ax[dti, c].set_ylim(0, np.percentile(plottingData.loc[:, 'count'].values, 98))
            
            # Set X axis to shared range
            mn, mx = plottingData.loc[(plottingData.human_or_stim_A==pA)&(
                plottingData.randomized_or_observed.isin(
                    ['randomized', 'observed'])), 'difference_histogram_bin'].agg(['min', 'max']).values
            ax[dti, c].set_xlim(mn, mx)
        
        for c in range(len(participantsSorted) - dt, len(participantsSorted) - 1):
            try:
                ax[dti, c].set_axis_off()
            except:
                pass

    fig.tight_layout()
    fig.subplots_adjust(left=0.06, bottom=0.1, right=0.94, top=0.94, wspace=0.25, hspace=0.25)
    fig.savefig('Landings_Permutation_Test_{}_{}_{}.pdf'.format(
        EXPERIMENT, MODE, 'absolutecounts' if USE_ABSOLUTE_COUNTS else 'percentages'))

In [ ]:
plotRankingTests(plottingData, MODE)